In [18]:
import sys
sys.path.append('/Users/yujunwon/Project/dancify/ai/yujun/complete')           # 경로 설정 필요

import angle_calculation

---
### 1. `angle_calculation.py`의 `compute_vector_angle` 함수의 boundary condition 검증

In [2]:
import numpy as np

L1 = np.array([1,1]) - np.array([2,1])
L2 = np.array([1,1]) - np.array([0,1])

# 벡터 크기
magnitude1 = np.linalg.norm(L1)
magnitude2 = np.linalg.norm(L2)

# 각도를 radian과 degree로 계산
angle_rad = np.arccos(np.dot(L1, L2) / (magnitude1 * magnitude2))
angle_deg = np.degrees(angle_rad)

print(angle_rad)
print(angle_deg)

3.141592653589793
180.0


---
### 2. JSON 비교 함수 구현에서 고려할 점
1. dancer_joint와 danceable_joint 중에서 하나에 맞춰서 비교해야 함 → 둘 중에서 짧은 것에 맞춰서 일단 구현
2. 프레임 단위로 비교할 때, 앞뒤 5프레임 정도를 평균내서 하는 것은 어떨지에 대해서 생각해보기 → 폐기

- 현재 `calculate_angle_difference()`는 두 JSON 중에서 짧은 것에 대응되도록 구현함

In [3]:
len(angle_calculation.calculate_angle_difference("../point_sample/test_data_1.json", "../point_sample/test_data_2.json"))

# test_data.json과 test_data_2.json은 각각 602, 601의 길이를 갖는다.
# 현재 구현된 함수에서는 의도적으로 프레임 수가 동일하지 않은 경우에서도 동작하도록 구현했다.

601

---
### 3. `calculation_angle_difference()`의 출력을 받아 부위별 평균 각도차를 점수로 환산
##### → `average_angle_difference()`

In [4]:
angle_calculation.average_angle_difference("../point_sample/test_data_1.json", "../point_sample/test_data_2.json")

[0.0, 0.0, 0.0, 0.0, 0.0, 0.016416524672587994, 0.0, 0.0]

In [5]:
def average_angle_difference(dancer_json_path: str, danceable_json_path: str):
    """
    두 JSON 파일의 키포인트 정보를 이용하여 각 관절 각도의 평균 차이를 계산합니다.

    Args:
        dancer_json_path (str): 댄서의 JSON 파일 경로
        danceable_json_path (str): 댄서블의 JSON 파일 경로

    Returns:
        List[float]: 각 관절 각도의 평균 차이를 담고 있는 리스트.
                     리스트의 각 요소는 다음의 관절을 나타냅니다:
                     0: 왼쪽 골반, 1: 오른쪽 골반, 2: 왼쪽 어깨, 3: 오른쪽 어깨,
                     4: 왼쪽 팔꿈치, 5: 오른쪽 팔꿈치, 6: 왼쪽 무릎, 7: 오른쪽 무릎

    Raises:
        ValueError: `difference` 리스트가 비어 있거나, `difference` 리스트의 길이가 0인 경우 발생
    """
    difference = angle_calculation.calculate_angle_difference(
        dancer_json_path, danceable_json_path)

    # difference 리스트가 비어 있는지 확인
    if not difference:
        raise ValueError("The difference list is empty.")

    angle_sums = [0.0] * 8

    for diff in difference:
        for i in range(len(diff)):
            angle_sums[i] += diff[i]

    # difference 리스트의 길이가 0인지 확인 (0으로 나누는 오류 방지)
    if len(difference) == 0:
        raise ValueError("Cannot divide by zero.")

    angle_difference = [x / len(difference) for x in angle_sums]

    return angle_difference


In [6]:
res = angle_calculation.average_angle_difference("../point_sample/danceable.json", "../point_sample/dancer.json")

In [16]:
difference = angle_calculation.calculate_angle_difference("../point_sample/danceable.json", "../point_sample/dancer.json")

# difference 리스트가 비어 있는지 확인
if not difference:
    raise ValueError("The difference list is empty.")

angle_sums = [0.0] * 8

for diff in difference:
    for i in range(len(diff)):
        angle_sums[i] += diff[i]

# difference 리스트의 길이가 0인지 확인 (0으로 나누는 오류 방지)
if len(difference) == 0:
    raise ValueError("Cannot divide by zero.")

angle_difference = [x / len(difference) for x in angle_sums]

difference_list = ["pelvis_diff", "shoulder_diff", "arm_diff", "leg_diff"]

result = {}
for idx in range(0, 8, 2):
    result[difference_list[idx//2]] = sum(res[idx:idx+2]) / len(difference)

result

# 이거 작성중!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

{'pelvis_diff': 10.555509695390388,
 'shoulder_diff': 12.528189692535157,
 'arm_diff': 21.230551312923787,
 'leg_diff': 36.057822399950815}

In [17]:
146.831 - 132.4199519412412

14.41104805875878